# 图像增强

图像增强（Image Emhancement）指对图像的亮度、对比度、色彩等属性进行调节，以改善视觉效果，便于人眼观察的相关技术和方法的统称。

但是值得注意的是，那些面向确定的、原因已知的图像退化的改善方法、去掉运动模糊或椒盐噪声、人为修改特定结构特征的方法，一般不认为是图像增强的范畴。

常用的图像增强算法主要分为基于点运算的方法和基于统计特性的方法。

## 基于点运算的方法

基于点运算得分方法实际上是按照一定的规则逐点改变像素的信号强度。先来观察这样一幅图像：

In [ ]:
from imgtools.utils import ASSETS_DIR
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np

imat_0 = np.array(Image.open(ASSETS_DIR / '01_low-contrast-pollen.png'))

print(f'Max pixel: {np.max(imat_0)}, Min pixel: {np.min(imat_0)}')
plt.imshow(imat_0, cmap='gray', vmin=0, vmax=255)
plt.show()

可以看到，它最大的像素值为138，最小的为91。但是一般灰度图像的像素值范围可取$[0, 255]$，所以，可以尝试把最大像素值都放大至255，或者把最小的像素值减小至0，观察图像发生的变化:

- 像素值增大，也就意味着亮度提升，使得原本暗处的细节更为明显，但是图像的对比度下降
- 像素值减小，亮度减弱，暗处的细节会更不清晰，但是主观上感觉对比度会提高，不同区域的对比和层次更明显

In [ ]:
plt.subplot(1, 3, 1)
plt.imshow(imat_0, cmap='gray', vmin=0, vmax=255)
plt.title('original')
plt.subplot(1, 3, 2)
plt.imshow(imat_0 + 117, cmap='gray', vmin=0, vmax=255)
plt.title('max = 255')
plt.subplot(1, 3, 3)
plt.imshow(imat_0 - 91 , cmap='gray', vmin=0, vmax=255)
plt.title('min = 0')
plt.show()

结合上面的思考，应该作两方面的考虑，即寻找一种方法，既能降低暗处的像素值，又可以提高亮处的像素值，兼顾亮度和对比度。考虑到基于点运算的方法，变换的输入和输出仅与像素值有关，而与像素值的空域位置无关，因此可以考虑简单的函数变换。

### 线性变换

线性变换可以表示为

$$
s = T(r) = kr + b \tag1
$$

为了最大限度实现上文的想法，预设了像素值的目标范围是$[s_1, s_2]$，因此可以简单求解出直线的斜率$k$和截距$b$（假设原图像的像素值范围是$[r_1, r_2]$）：

$$
\begin{cases}
k = \frac {s_2 - s_1} {r_2 - r_1} \\
b = s_1 - kr_1
\end{cases} \tag2
$$

代入上式即可得到下面的代码：

In [ ]:
r1, r2 = np.min(imat_0), np.max(imat_0)
s1, s2 = 0, 255

k = (s2 - s1) / (r2 - r1)
b = s1 - k * r1

def transform(x):
    return k * x + b

imat_tmp= transform(imat_0)
imat_linear_transformed = np.array(imat_tmp, dtype=np.uint8)

r = np.linspace(r1, r2, r2 - r1 + 1)
s = transform(r)

plt.subplot(1, 3, 1)
plt.imshow(imat_0, cmap='gray', vmin=0, vmax=255)
plt.title('original')
plt.subplot(1, 3, 2)
plt.imshow(imat_linear_transformed, cmap='gray', vmin=0, vmax=255)
plt.title('transformed')
plt.subplot(1, 3, 3)
plt.plot(r, s)
plt.title('transformation')
plt.show()